In [1]:
# create and solve basic equilibrium.
# taking Example_1 and putting it all here to use as eq for testing probes


import os
os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils_from_UDA.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"
# the following environment variable is optional, 
# it contains data on an equilibrium that is representative for the tokamak 
# for more info please refer to Example_0
os.environ["EQUILIBRIUM_PATH"] = "../machine_configs/MAST-U/simple_initilization_equilibrium.pickle"
os.environ["PROBE_PATH"] = "../machine_configs/MAST-U/magnetic_probes.pickle"
# Now the machine can actually be built:
from freegsnke import build_machine
tokamak = build_machine.tokamak()

from freegsnke import equilibrium_update
from freegsnke.jtor_update import ConstrainPaxisIp
from freegsnke import GSstaticsolver


# seems to need to add in import freegs?
import freegs

In [2]:
# create equilibirum object
eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,
    Rmin=0.1, Rmax=2.0,   # Radial range
    Zmin=-2.2, Zmax=2.2,  # Vertical range
    nx=65,   # Number of grid points in the radial direction
    ny=129,  # Number of grid points in the vertical direction
    # note that both nx and ny have to be of the form 2**i + 1 with i being an integer
    # psi=plasma_psi
)  


# create profile object
profiles = ConstrainPaxisIp(
    eq=eq,
    limiter=tokamak.limiter,
    paxis=8e3,
    Ip=6e5,
    fvac=0.5,
    alpha_m=1.8,
    alpha_n=1.2
)

Equilibrium initialised using file provided as part of the machine description.


In [3]:
# load nonlinear solver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)

# setup initial conditions....
# Set X-point coordinates
Rx = 0.6
Zx = 1.1
xpoints = [(Rx, -Zx),   
           (Rx,  Zx)]

# Set any desired isoflux constraints,
# these have structure (R1, Z1, R2, Z2)
# where (R1, Z1) and (R2, Z2) are 
# desired to be on the same flux contour.
Rmid = 1.4   # Outboard midplane
Rin = 0.35    # Inboard midplane
isoflux = [(Rx,Zx, Rx,-Zx), # link X-points
           (Rmid, 0, Rin, 0.0), # link inner and outer midplane points
           (Rmid, 0, Rx, Zx), # link outer midplane point and X-point

        #   some additional constraints
           (Rmid,0, 1.2,.7),
           (Rmid,0, 1.2,-.7),
           (Rx, Zx, .85, 1.7),
           (Rx, Zx, .75, 1.6),
           (Rx, Zx, Rin, 0.2),
           (Rx, Zx, Rin, 0.1),
           (Rx,-Zx, Rin, -0.1),
           (Rx,-Zx, Rin, -0.2),
           (Rx,-Zx, .85, -1.7),
           (Rx,-Zx, .75, -1.6),
           (Rx,-Zx, 0.45, -1.8),
           (Rx, Zx, 0.45,  1.8),
           ]
           
# Instantiate the constrain object
constrain = freegs.control.constrain(xpoints=xpoints, 
                                     isoflux=isoflux,
                                     gamma=5e-7
                                    )

# apply solver
GSStaticSolver.solve(eq=eq, 
                     profiles=profiles, 
                     constrain=constrain, 
                     target_relative_tolerance=1e-3)

AttributeError: 'super' object has no attribute 'Jtor_part1'

This is only the inverse solve, but will have coil and plasma currents that can be used by the probes.

Let's set this up now.

In [4]:
from freegsnke import magnetic_probes

flux_probe = magnetic_probes.probe(eq)
print(len(flux_probe.floops))
# print(flux_probe.floop_pos)
# print(flux_probe.floop_pos_R)
print(flux_probe.floop_pos_R.shape)
print("---------")
print(flux_probe.greens_psi_single_coil('Solenoid'))
print(flux_probe.greens_psi_all_coils())
print(len(flux_probe.get_coil_currents(eq)))
print(len(flux_probe.psi_all_coils(eq)))
# print(len(flux_probe.psi_from_plasma(eq)))


102
(102,)
---------
[1.4698765e-06 1.5152706e-06 1.5719672e-06 1.6074496e-06 1.5386220e-06
 1.5363846e-06 1.6332915e-06 1.4690336e-06 1.5145058e-06 1.5712152e-06
 1.6067950e-06 1.5380346e-06 1.5357994e-06 1.6351767e-06 1.6980265e-06
 1.6939957e-06 1.1565966e-06 1.1516602e-06 1.1398154e-06 1.1221541e-06
 1.1564298e-06 1.1515172e-06 1.1396945e-06 1.1216772e-06 1.7421148e-06
 1.7558443e-06 1.7676070e-06 1.7774148e-06 1.7765312e-06 1.7665304e-06
 1.7545744e-06 1.7406492e-06 9.1217447e-07 8.1241899e-07 7.3560250e-07
 6.8900283e-07 6.3837831e-07 6.2960299e-07 7.0954616e-07 9.0162769e-07
 8.0461803e-07 7.2981635e-07 6.8437276e-07 6.3493042e-07 6.2751133e-07
 7.0925353e-07 1.2506252e-06 1.3293022e-06 1.2356329e-06 1.3046495e-06
 1.3038361e-06 1.3283736e-06 1.2348124e-06 1.2496864e-06 5.5973942e-07
 6.1908179e-07 6.1689803e-07 6.7398139e-07 6.1242957e-07 6.6972518e-07
 5.5569592e-07 6.1515573e-07 8.0238277e-07 7.2102074e-07 9.0302393e-07
 8.3565766e-07 8.9174421e-07 8.2257878e-07 7.9207626e-07